In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.getcwd()
os.chdir('drive')
os.chdir('My Drive')
os.chdir('Janata')
os.listdir()
# os.listdir()

['train.csv',
 'test.csv',
 'submission2.csv',
 'submission3.csv',
 'submission4.csv',
 'submission5.csv',
 'Janata_DNN.ipynb']

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

newdf = pd.read_csv('train.csv')

appendvr2newdf = pd.get_dummies(newdf['var2'],drop_first=True)
newdf = pd.concat([newdf.drop('var2',axis=1),appendvr2newdf], axis=1)

newdf['datetime'] = pd.to_datetime(newdf['datetime'])

newdf['day'] = newdf['datetime'].dt.day
newdf['month'] = newdf['datetime'].dt.month
newdf['year'] = newdf['datetime'].dt.year
newdf['time'] = newdf['datetime'].dt.time

newdf.drop('datetime',axis = 1,inplace = True)

newdf['hour'] = newdf['time'].apply(lambda x : x.hour)
newdf.drop('time',axis =1 , inplace = True)

appendvr2newdf = pd.get_dummies(newdf['month'],drop_first=True)
newdf = pd.concat([newdf.drop('month',axis=1),appendvr2newdf], axis=1)

appendvr2newdf = pd.get_dummies(newdf['year'],drop_first=True)
newdf = pd.concat([newdf.drop('year',axis=1),appendvr2newdf], axis=1)

appendvr2newdf = pd.get_dummies(newdf['hour'],drop_first=True)
newdf = pd.concat([newdf.drop('hour',axis=1),appendvr2newdf], axis=1)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from sklearn.model_selection import train_test_split

X = newdf.drop(['ID','electricity_consumption'],axis = 1)
y = newdf['electricity_consumption']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [8]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(45, activation='relu',input_dim = X_train.shape[1]))

# The Hidden Layers :
NN_model.add(Dense(45, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(45, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(45, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 45)                2070      
_________________________________________________________________
dense_1 (Dense)              (None, 45)                2070      
_________________________________________________________________
dense_2 (Dense)              (None, 45)                2070      
_________________________________________________________________
dense_3 (Dense)              (None, 45)                2070      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 46        
Total params: 8,326
Trainable params: 8,326
Non-trainable params: 0
_________________________________________________________________


In [9]:
import tensorflow as tf

es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience =10, mode='min')

NN_model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=[es])

Epoch 1/500
464/464 [==============================] - 1s 3ms/step - loss: 106.9160 - mean_absolute_error: 106.9441 - val_loss: 66.0519 - val_mean_absolute_error: 66.0515
Epoch 2/500
464/464 [==============================] - 1s 2ms/step - loss: 64.0145 - mean_absolute_error: 64.0108 - val_loss: 61.2080 - val_mean_absolute_error: 61.2055
Epoch 3/500
464/464 [==============================] - 1s 2ms/step - loss: 60.2585 - mean_absolute_error: 60.2643 - val_loss: 58.9071 - val_mean_absolute_error: 58.9037
Epoch 4/500
464/464 [==============================] - 1s 2ms/step - loss: 57.8901 - mean_absolute_error: 57.8675 - val_loss: 57.6404 - val_mean_absolute_error: 57.6368
Epoch 5/500
464/464 [==============================] - 1s 2ms/step - loss: 56.6786 - mean_absolute_error: 56.6806 - val_loss: 56.3848 - val_mean_absolute_error: 56.3793
Epoch 6/500
464/464 [==============================] - 1s 3ms/step - loss: 55.4068 - mean_absolute_error: 55.3864 - val_loss: 56.3474 - val_mean_absolute

In [10]:
predictions = NN_model.predict(X_test)

from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

RMSE: 74.2114688128947


In [0]:
testdf = pd.read_csv('test.csv')

appendvr2testdf = pd.get_dummies(testdf['var2'],drop_first=True)
testdf = pd.concat([testdf.drop('var2',axis=1),appendvr2testdf], axis=1)

testdf['datetime'] = pd.to_datetime(testdf['datetime'])

testdf['day'] = testdf['datetime'].dt.day
testdf['month'] = testdf['datetime'].dt.month
testdf['year'] = testdf['datetime'].dt.year
testdf['time'] = testdf['datetime'].dt.time

testdf.drop('datetime',axis = 1,inplace = True)

testdf['hour'] = testdf['time'].apply(lambda x : x.hour)
testdf.drop('time',axis =1 , inplace = True)


appendvr2testdf = pd.get_dummies(testdf['month'],drop_first=True)
testdf = pd.concat([testdf.drop('month',axis=1),appendvr2testdf], axis=1)
appendvr2testdf = pd.get_dummies(testdf['year'],drop_first=True)
testdf = pd.concat([testdf.drop('year',axis=1),appendvr2testdf], axis=1)
appendvr2testdf = pd.get_dummies(testdf['hour'],drop_first=True)
testdf = pd.concat([testdf.drop('hour',axis=1),appendvr2testdf], axis=1)

In [0]:
X_test = testdf.drop(['ID'],axis = 1)
X_test = scaler.transform(X_test)

In [0]:
predictionstest = NN_model.predict(X_test)

In [0]:
submission = pd.DataFrame(predictionstest,columns=['electricity_consumption'])

finalsubmission = pd.concat([testdf[['ID']],submission],axis=1)

finalsubmission.to_csv('submission5.csv',index=False)